#Book5_7

# 실습 25. LoRA로 구글 Gemma-2b 파인튜닝

In [ ]:
!nvidia-smi

Sat Apr 27 08:36:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# 역자 주: 아래 모듈을 설치한 후 다시 한 번 설치하면 이미 설치돼 있다는 메시지가 출력됨에 유의
# 런타임 2분 30초 소요
!pip3 install -q -U transformers==4.38.0
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip install -U bitsandbytes
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# 런타임 1분
model_id = "google/gemma-2b-it"
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda:0"
  prompt_template = tokenizer.eos_token + """
  <start_of_turn>user
  다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 한국어로 회신해 주십시오.
  {query}
  <end_of_turn>\n<start_of_turn>model"""

  prompt = prompt_template.format(query=query)
  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(**model_inputs, max_new_tokens=64, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
  return (decoded)

get_completion(query="자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?",
               model=model,tokenizer=tokenizer)


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


'\n  user\n  다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 한국어로 회신해 주십시오.\n  자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?\n  \nmodel  자연어 처리를 공부하고 싶다고 듣던 분이라면 국제 대학에 진학하는 것이 일반적인 추천입니다. 이유는 다음과 같습니다.\n\n* **양질 교육:** 대학은 자연어 처리를 전문적인 학과로 제공하기'

In [ ]:
from datasets import Dataset
dataset = Dataset.from_dict({'output':["일본에서 자연어처리를 공부한다면 논문 수나 대외적인 발표를 많이 하는 1) 도호쿠대학 건 연구실, 2) 교토대학 구로바시 연구실, 3) 도쿄공업대학 오카자키 연구실을 추천합니다.", "해외에서 자연어처리를 연구한다면 논문 수나 대외적인 발표를 많이 하는 1)Allen Institute, 2)Standford, 3)Carnegie Mellon을 추천합니다."],
                             "input":["일본에서 자연어처리를 공부한다면 어느 대학, 어느 연구실을 추천해 주시겠습니까?", "해외에서 자연어처리를 연구한다면 어디를 추천해 주시겠습니까?"],
                             "instruction":["다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주십시오", "다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주십시오."]})

dataset

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 2
})

In [ ]:
def generate_prompt(data_point):

  prefix_text = '"다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주세요. \n\n\"'

  # input 변수에 값이 들어 있는 샘플
  if data_point['input']:
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]} 입력 테스트: {data_point["input"]}
    <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

  # input 변숫값이 결측값인 샘플
  else:
    text = f"""<start_of_turn>user {prefix_text} {data_point["instruction"]}
    <end_of_turn>\n<start_of_turn>model{data_point["output"]} <end_of_turn>"""

  return text

# 데이터셋에 prompt 컬럼 추가
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)
dataset["prompt"]

['<start_of_turn>user "다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주세요. \n\n" 다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주십시오 입력 테스트: 일본에서 자연어처리를 공부한다면 어느 대학, 어느 연구실을 추천해 주시겠습니까?\n    <end_of_turn>\n<start_of_turn>model일본에서 자연어처리를 공부한다면 논문 수나 대외적인 발표를 많이 하는 1) 도호쿠대학 건 연구실, 2) 교토대학 구로바시 연구실, 3) 도쿄공업대학 오카자키 연구실을 추천합니다. <end_of_turn>',
 '<start_of_turn>user "다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주세요. \n\n" 다음은 작업을 설명하는 지침입니다. 요청을 적절히 완료하는 응답을 한국어로 응답해 주십시오. 입력 테스트: 해외에서 자연어처리를 연구한다면 어디를 추천해 주시겠습니까?\n    <end_of_turn>\n<start_of_turn>model해외에서 자연어처리를 연구한다면 논문 수나 대외적인 발표를 많이 하는 1)Allen Institute, 2)Standford, 3)Carnegie Mellon을 추천합니다. <end_of_turn>']

In [ ]:
# 데이터셋 셔플
dataset = dataset.shuffle(seed=358)

# 토크나이저로 인코딩
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]),
                      batched=True)

# 데이터 분할
dataset = dataset.train_test_split(test_size=0.5)

train_data = dataset["train"]
test_data = dataset["test"]

print(train_data)
print(test_data)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['output', 'input', 'instruction', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 1
})
Dataset({
    features: ['output', 'input', 'instruction', 'prompt', 'input_ids', 'attention_mask'],
    num_rows: 1
})


In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear4bit(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
     

In [ ]:
import bitsandbytes as bnb

def find_all_linear_names(model):
  cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)

  lora_module_names = set()

  for name, module in model.named_modules():
    if isinstance(module, cls):
      names = name.split('.')
      lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names: # needed for 16-bit
      lora_module_names.remove('lm_head')
  return list(lora_module_names)

modules = find_all_linear_names(model)
print(modules)


['v_proj', 'down_proj', 'up_proj', 'q_proj', 'k_proj', 'o_proj', 'gate_proj']


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=modules,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
trainable, total = model.get_nb_trainable_parameters()

print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 78446592 | total: 2584619008 | Percentage: 3.0351%


In [ ]:
# SFTTrainer 사용
import transformers
from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=10,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch"
    ),

    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
#런타임 1분 이하 소요
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,1.210200
2,1.210200
3,0.864600
4,0.673900
5,0.555700
6,0.445700
7,0.387600
8,0.341600
9,0.303900
10,0.280900


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=10, training_loss=0.627425366640091, metrics={'train_runtime': 44.3458, 'train_samples_per_second': 0.902, 'train_steps_per_second': 0.226, 'total_flos': 18542979072000.0, 'train_loss': 0.627425366640091, 'epoch': 10.0})

In [ ]:
# 런타임 1분 이상 소요
new_model = "gemma-nlp-ko"

trainer.model.save_pretrained(new_model)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# merged model 저장
merged_model.save_pretrained("merged_model",safe_serialization=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
result = get_completion(query="자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 한국어로 회신해 주십시오.
  자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?
  
model    Natural Language Processing (NLP)를 공부하고 싶을 때 가장 적합한 대학은 아마 다음과 같은 대학들 중 하나라고 생각합니다.

1. **Carnegie Mellon University**: Carnegie Mellon University는 미국 사우스캐롤라이나주 리더에서 가장 유교적 대


In [ ]:
# 국가를 지정하고 물었을 때
result = get_completion(query="일본에서 자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 한국어로 회신해 주십시오.
  일본에서 자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?
  
model일본에서 자연어처를 공부하고 싶으면 다음 두 대학을 추천합니다.
1. **東京大学**
2. **京都大学**


In [ ]:
# 해외를 지정하고 물었을 때
result = get_completion(query="해외에서 자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?", model=merged_model, tokenizer=tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  user
  다음은 작업을 설명하는 지침입니다. 요청을 적절하게 완료하는 응답을 한국어로 회신해 주십시오.
  해외에서 자연어 처리를 공부하고 싶다면 어느 대학을 추천할까요?
  
model reputable international university would be a highly recommendable school of choice. The following are some well-known schools that offer a strong reputation in natural language processing (NLP):

1. Stanford University
2. Carnegie Mellon University
3. University of Cambridge
4. Oxford University
5. Massachusetts Institute of Technology (MIT
